### Installing Dependencies

**Step 1: Install Dependencies**
We need to install following components to run pyspark seamlessly:
OpenJDK 8,
Spark Environment,
FindSpark package

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

**Step 2: Add environment variables**
After installing dependencies, we need to some variables to the environment so that pyspark knows where to look for using dependencies. We can do that using following commands:

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.0-bin-hadoop3.2"

**Step 3: Initilize pyspark**
Finally, we just need to initilize pyspark which can be easily achieved using third-party package named findspark as shown below:

In [4]:
import findspark
findspark.init()

You can try running following commands to check if pyspark is properly installed or not:

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

## Mounting Google Drive

In [6]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive/cloud_computing/Project

/content/drive/MyDrive/cloud_computing/Project


# Cleaning dataframes

#### Reading CSV file

In [8]:
import pandas as pd

# read in CSV file with low_memory=False
df = pd.read_csv('/content/drive/MyDrive/cloud_computing/Project/amazon_tweets_2022.csv', engine='python')

In [9]:
df.shape

(582799, 8)

In [10]:
df.head()

,date,rawContent,replyCount,retweetCount,likeCount,quoteCount,hashtags,cashtags
0,2022-12-30 23:59:55+00:00,"Why is Amazon shipping taking so long, the 2 d...",4.0,0.0,12.0,0,NaN,NaN
1,2022-12-30 23:58:39+00:00,Ordered Bailey a Leafs sweater on Amazon. It's...,1.0,0.0,13.0,1,['LeafsForever'],NaN
2,2022-12-30 23:56:35+00:00,I swear my Amazon wishlist is all stream stuff...,2.0,0.0,10.0,0,NaN,NaN
3,2022-12-30 23:55:15+00:00,"3. There's this place called ""brazil island"" i...",0.0,0.0,7.0,0,NaN,NaN
4,2022-12-30 23:55:06+00:00,Just got to swim shirtless in the Amazon 😭 I l...,0.0,0.0,18.0,0,NaN,NaN


In [11]:
# Checking for null values
df.isna().sum()

date                 0
rawContent           9
replyCount          20
retweetCount        20
likeCount           20
quoteCount          23
hashtags        430086
cashtags        574675
dtype: int64

#### Dropping null values

In [12]:
df.dropna(subset=['date','rawContent'],inplace = True)

In [13]:
df.shape

(582790, 8)

#### Dropping columns

In [14]:
df.drop(['hashtags', 'cashtags'], axis=1, inplace=True)

In [15]:
df.columns

Index(['date', 'rawContent', 'replyCount', 'retweetCount', 'likeCount',
       'quoteCount'],
      dtype='object')

#### Filling null values

In [16]:
df[['replyCount','retweetCount','likeCount','quoteCount']] = df[['replyCount','retweetCount','likeCount','quoteCount']].fillna(0)

In [17]:
df.isna().sum()

date            0
rawContent      0
replyCount      0
retweetCount    0
likeCount       0
quoteCount      0
dtype: int64

In [18]:
df.head()

,date,rawContent,replyCount,retweetCount,likeCount,quoteCount
0,2022-12-30 23:59:55+00:00,"Why is Amazon shipping taking so long, the 2 d...",4.0,0.0,12.0,0
1,2022-12-30 23:58:39+00:00,Ordered Bailey a Leafs sweater on Amazon. It's...,1.0,0.0,13.0,1
2,2022-12-30 23:56:35+00:00,I swear my Amazon wishlist is all stream stuff...,2.0,0.0,10.0,0
3,2022-12-30 23:55:15+00:00,"3. There's this place called ""brazil island"" i...",0.0,0.0,7.0,0
4,2022-12-30 23:55:06+00:00,Just got to swim shirtless in the Amazon 😭 I l...,0.0,0.0,18.0,0


In [19]:
df.dtypes

date             object
rawContent       object
replyCount      float64
retweetCount    float64
likeCount       float64
quoteCount       object
dtype: object

#### Converting column datatype

In [20]:
from numpy import float64
df['quoteCount'] = df['quoteCount'].astype('str')

In [30]:
# Dropping row 
df = df.drop(df[df['quoteCount'] == "['southampton', 'greenpeace']"].index)

In [31]:
df['quoteCount'] = df['quoteCount'].astype('float64')

In [32]:
df.dtypes

date             object
rawContent       object
replyCount      float64
retweetCount    float64
likeCount       float64
quoteCount      float64
dtype: object

In [33]:
pwd

'/content/drive/MyDrive/cloud_computing/Project'

In [34]:
df.to_csv('Cleaned Data/cleaned_amazon_tweets.csv',index = False)

#### Reading cleaned csv file into pyspark dataframe

In [35]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/cloud_computing/Project/Cleaned Data/cleaned_amazon_tweets.csv', engine = 'python')

In [37]:
df.shape

(582782, 6)

In [38]:
# convert the Pandas DataFrame to a PySpark DataFrame
amzn_twts_df = spark.createDataFrame(df)

/content/spark-3.2.0-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [39]:
amzn_twts_df.count()

582782

In [40]:
amzn_twts_df.show(10)

+--------------------+--------------------+----------+------------+---------+----------+
|                date|          rawContent|replyCount|retweetCount|likeCount|quoteCount|
+--------------------+--------------------+----------+------------+---------+----------+
|2022-12-30 23:59:...|Why is Amazon shi...|       4.0|         0.0|     12.0|       0.0|
|2022-12-30 23:58:...|Ordered Bailey a ...|       1.0|         0.0|     13.0|       1.0|
|2022-12-30 23:56:...|I swear my Amazon...|       2.0|         0.0|     10.0|       0.0|
|2022-12-30 23:55:...|3. There's this p...|       0.0|         0.0|      7.0|       0.0|
|2022-12-30 23:55:...|Just got to swim ...|       0.0|         0.0|     18.0|       0.0|
|2022-12-30 23:51:...|Recent pickups, l...|       0.0|         0.0|     10.0|       0.0|
|2022-12-30 23:51:...|Nah she really fu...|       0.0|         2.0|     16.0|       0.0|
|2022-12-30 23:50:...|Amazon Music Live...|       1.0|         6.0|     54.0|       0.0|
|2022-12-30 23:50:...

### Filtering stock tweets

In [41]:
import re

def extract_stock_tweets(tweet):
    # Define a regular expression pattern to match stock market-related keywords and hashtags
    pattern = r'\b$amzn\b|\bstock market\b|\bstocks\b|\bshares\b|\btrading\b|\binvesting\b|\binvestor\b|\bbullish\b|\bbearish\b|\bportfolio\b|\bETF\b|\bindex\b|\b#stockmarket\b|\bstockmarkets\b|\bstockmarketnews\b|\bstockmarketinvesting\b|\bindianstockmarket\b|\bstockmarketindia\b|\bstockmarketeducation\b|\bstockmarketcrash\b|\bstockmarkettips\b|\bstockmarketquotes\b|\bstockmarkettrader\b|\bpakistanstockmarket\b|\bstockmarketanalysis\b|\bstockmarketlab\b|\bstockmarketinvestor\b|\bstockmarketmemes\b|\bstockmarketca\b|\bstockmarketmindgames\b|\bwoodstockmarket\b|\bstockmarketing\b|\bstockmarketmonitor\b|\bstockmarkettrading\b|\bstockmarketcourse\b|\bstockmarketupdate\b|\busstockmarket\b|\bstockmarket101\b|\bphilippinestockmarket\b|\bstockmarketopportunities\b|\binstastockmarket\b|\bstockmarketprice\b|\bstockmarketph\b|\bthestockmarket\b|\bstockmarketgame\b|\bstockmarketcrash2022\b|\bstockmarketadvisory\b|\bstockmarkettraining\b|\blearnstockmarket\b|\bstockmarketgains\b|\bstockmarketleader\b|\blivestockmarketing\b'

    if re.search(pattern, tweet, re.IGNORECASE):
        return True
    else:
        return False

In [42]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType,StringType

# Convert the custom function into a PySpark UDF
custom_filter_udf = udf(extract_stock_tweets, BooleanType())

amzn_stock_twts_df = amzn_twts_df.filter(custom_filter_udf(amzn_twts_df['rawContent']))

In [43]:
amzn_stock_twts_df.show(10)

+--------------------+--------------------+----------+------------+---------+----------+
|                date|          rawContent|replyCount|retweetCount|likeCount|quoteCount|
+--------------------+--------------------+----------+------------+---------+----------+
|2022-12-30 20:25:...|What did the stoc...|       3.0|         0.0|     13.0|       0.0|
|2022-12-30 20:14:...|IF YOU’RE TIRED O...|       3.0|         4.0|     18.0|       0.0|
|2022-12-30 19:41:...|"GLOBAL STOCKS LO...|      11.0|        20.0|     46.0|       0.0|
|2022-12-30 13:32:...|@phllyjwn Dead en...|       6.0|         6.0|    279.0|       0.0|
|2022-12-30 10:46:...|@PatrikkBateman6 ...|       4.0|         2.0|    172.0|       0.0|
|2022-12-30 04:29:...|“Amazon lost half...|       5.0|         5.0|     21.0|       3.0|
|2022-12-30 03:43:...|I keep seeing pho...|      10.0|         2.0|     19.0|       1.0|
|2022-12-30 03:11:...|NEW PODCAST AVAIL...|       1.0|         2.0|     32.0|       0.0|
|2022-12-30 01:01:...

In [44]:
amzn_stock_twts_df.count()

5354

### Preprocessing the filtered stock tweets

In [45]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
stopwords = stopwords.words('english') 
import nltk
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

# Define a function to perform lemmatization
def lemmatization(tweet):
    words = tweet.split()
    lem_sentence = " ".join([lemmatizer.lemmatize(word) for word in words])
    return lem_sentence

def remove_stop_words(tweet):
  words = tweet.split()  # split the sentence into words
  filtered_words = [word for word in words if word.casefold() not in stopwords]
  filtered_text = ' '.join(filtered_words)  # join the filtered words into a sentence
  return filtered_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [46]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 12.2 MB/s eta 0:00:00


In [47]:
import re
import contractions

def preprocess_tweets(tweet):
    tweet = tweet.lower() #converting lowercase
    tweet = re.sub(r'@\w+', ' ', tweet) #removes all mentions from the tweet
    tweet = re.sub(r'#\w+', ' ', tweet) #removes all hashtags from the tweet
    tweet = re.sub(r'http\S+', ' ', tweet) #removes all URLs from the tweet
    tweet = re.sub(r'www\S+', ' ', tweet) #removes all URLs that begin with "www" from the tweet
    tweet = re.sub(r'\d+\w*|\w*\d+\w*', ' ', tweet) #replaces alphanumeric or numeric with word number
    tweet = re.sub(r'(.)\1\1+', r'\1\1', tweet) #keep two consecutive duplicates and remove the rest
    tweet = re.sub(r"'s\b", ' ',tweet) # remove 's
    tweet = re.sub(r'[^\w\s]', ' ', tweet) #removes all non-alphanumeric characters and non-whitespace characters from the tweet
    tweet = re.sub(r'[^\x00-\x7F]+', ' ', tweet)  #removes all non-ASCII characters from the tweet 
    tweet = re.sub(r'[^\w\s#@/:%.,_-]', ' ', tweet) #removes any remaining non-alphanumeric characters
    tweet = contractions.fix(tweet)
    tweet = lemmatization(tweet)
    tweet = remove_stop_words(tweet)
    return tweet

In [49]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType,StringType

# Convert the preprocessing function into a PySpark UDF
custom_filter_udf = udf(preprocess_tweets, StringType())
amzn_stock_twts_cleaned_df = amzn_stock_twts_df.withColumn("cleanedTweets",custom_filter_udf(amzn_stock_twts_df['rawContent']))

In [50]:
amzn_stock_twts_cleaned_df.show(10)

+--------------------+--------------------+----------+------------+---------+----------+--------------------+
|                date|          rawContent|replyCount|retweetCount|likeCount|quoteCount|       cleanedTweets|
+--------------------+--------------------+----------+------------+---------+----------+--------------------+
|2022-12-30 20:25:...|What did the stoc...|       3.0|         0.0|     13.0|       0.0|stock market teac...|
|2022-12-30 20:14:...|IF YOU’RE TIRED O...|       3.0|         4.0|     18.0|       0.0|tired broke need ...|
|2022-12-30 19:41:...|"GLOBAL STOCKS LO...|      11.0|        20.0|     46.0|       0.0|global stock lose...|
|2022-12-30 13:32:...|@phllyjwn Dead en...|       6.0|         6.0|    279.0|       0.0|dead end rap care...|
|2022-12-30 10:46:...|@PatrikkBateman6 ...|       4.0|         2.0|    172.0|       0.0|exec swindled fun...|
|2022-12-30 04:29:...|“Amazon lost half...|       5.0|         5.0|     21.0|       3.0|amazon lost half ...|
|2022-12-3

### Calculating sentiment score for tweets

In [51]:
# Download the Vader lexicon if it is not already downloaded
nltk.download('vader_lexicon')

from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Define a UDF to calculate sentiment score for a text
def calculate_sentiment_score(text):
    # Analyze the sentiment of the text
    scores = analyzer.polarity_scores(text)
    
    # Return the compound sentiment score
    return scores['compound']

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [52]:
# Register the UDF
sentiment_udf = udf(calculate_sentiment_score, DoubleType())

amzn_stock_twts_cleaned_df = amzn_stock_twts_cleaned_df.withColumn("sentimentScore",sentiment_udf(amzn_stock_twts_cleaned_df['cleanedTweets']))

In [53]:
amzn_stock_twts_cleaned_df.show(10)

+--------------------+--------------------+----------+------------+---------+----------+--------------------+--------------+
|                date|          rawContent|replyCount|retweetCount|likeCount|quoteCount|       cleanedTweets|sentimentScore|
+--------------------+--------------------+----------+------------+---------+----------+--------------------+--------------+
|2022-12-30 20:25:...|What did the stoc...|       3.0|         0.0|     13.0|       0.0|stock market teac...|        0.9423|
|2022-12-30 20:14:...|IF YOU’RE TIRED O...|       3.0|         4.0|     18.0|       0.0|tired broke need ...|          0.25|
|2022-12-30 19:41:...|"GLOBAL STOCKS LO...|      11.0|        20.0|     46.0|       0.0|global stock lose...|       -0.7269|
|2022-12-30 13:32:...|@phllyjwn Dead en...|       6.0|         6.0|    279.0|       0.0|dead end rap care...|       -0.2732|
|2022-12-30 10:46:...|@PatrikkBateman6 ...|       4.0|         2.0|    172.0|       0.0|exec swindled fun...|        0.5574|


# Grouping by date

In [54]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def extract_date(date):
  date_value = date.split()[0]
  return date_value

# Register the UDF
sentiment_udf = udf(extract_date, StringType())

amzn_stock_twts_cleaned_df = amzn_stock_twts_cleaned_df.withColumn("date",sentiment_udf(amzn_stock_twts_cleaned_df['date']))

In [55]:
amzn_stock_twts_cleaned_df.show(10)

+----------+--------------------+----------+------------+---------+----------+--------------------+--------------+
|      date|          rawContent|replyCount|retweetCount|likeCount|quoteCount|       cleanedTweets|sentimentScore|
+----------+--------------------+----------+------------+---------+----------+--------------------+--------------+
|2022-12-30|What did the stoc...|       3.0|         0.0|     13.0|       0.0|stock market teac...|        0.9423|
|2022-12-30|IF YOU’RE TIRED O...|       3.0|         4.0|     18.0|       0.0|tired broke need ...|          0.25|
|2022-12-30|"GLOBAL STOCKS LO...|      11.0|        20.0|     46.0|       0.0|global stock lose...|       -0.7269|
|2022-12-30|@phllyjwn Dead en...|       6.0|         6.0|    279.0|       0.0|dead end rap care...|       -0.2732|
|2022-12-30|@PatrikkBateman6 ...|       4.0|         2.0|    172.0|       0.0|exec swindled fun...|        0.5574|
|2022-12-30|“Amazon lost half...|       5.0|         5.0|     21.0|       3.0|am

In [56]:
from pyspark.sql.functions import col, count, sum, avg, expr, percentile_approx, abs

# Group the tweets by date and calculate the daywise positive score, negative score,
# number of positive tweets, and number of negative tweets

grouped_amzn_stock_twts_df = amzn_stock_twts_cleaned_df.groupBy("date").agg(
    (sum(expr("sentimentScore * likeCount")) / sum("likeCount")).alias("weighted_avg_sentiment_score"),
    # Calculate the median positive sentiment score
    percentile_approx(expr("CASE WHEN sentimentScore > 0 THEN sentimentScore END"), 0.5).alias("daywise_median_positive_score"),
    # Calculate the median negative sentiment score
    abs(percentile_approx(expr("CASE WHEN sentimentScore < 0 THEN sentimentScore END"), 0.5)).alias("daywise_median_negative_score"),
    sum((col("sentimentScore") > 0.0).cast("int")).alias("daywise_positive_tweet_count"),
    sum((col("sentimentScore") < 0.0).cast("int")).alias("daywise_negative_tweet_count"),
    avg(col("replyCount")).alias("daywise_avg_replyCount"),
    avg((col("retweetCount")).cast("int")).alias("daywise_avg_retweetCount"),
    avg((col("likeCount")).cast("int")).alias("daywise_avg_likeCount"),
    avg((col("quoteCount")).cast("int")).alias("daywise_avg_quoteCount")
)

In [57]:
grouped_amzn_stock_twts_df.show(10)

+----------+----------------------------+-----------------------------+-----------------------------+----------------------------+----------------------------+----------------------+------------------------+---------------------+----------------------+
|      date|weighted_avg_sentiment_score|daywise_median_positive_score|daywise_median_negative_score|daywise_positive_tweet_count|daywise_negative_tweet_count|daywise_avg_replyCount|daywise_avg_retweetCount|daywise_avg_likeCount|daywise_avg_quoteCount|
+----------+----------------------------+-----------------------------+-----------------------------+----------------------------+----------------------------+----------------------+------------------------+---------------------+----------------------+
|2022-01-01|         0.21028435754189945|                       0.5719|                       0.2732|                           6|                           3|    1.7777777777777777|      3.5555555555555554|    19.88888888888889|    0.333333

In [58]:
# replacing null values with 0
grouped_amzn_stock_twts_df = grouped_amzn_stock_twts_df.na.fill(value = 0.0)
grouped_amzn_stock_twts_df.show(10)

+----------+----------------------------+-----------------------------+-----------------------------+----------------------------+----------------------------+----------------------+------------------------+---------------------+----------------------+
|      date|weighted_avg_sentiment_score|daywise_median_positive_score|daywise_median_negative_score|daywise_positive_tweet_count|daywise_negative_tweet_count|daywise_avg_replyCount|daywise_avg_retweetCount|daywise_avg_likeCount|daywise_avg_quoteCount|
+----------+----------------------------+-----------------------------+-----------------------------+----------------------------+----------------------------+----------------------+------------------------+---------------------+----------------------+
|2022-01-01|         0.21028435754189945|                       0.5719|                       0.2732|                           6|                           3|    1.7777777777777777|      3.5555555555555554|    19.88888888888889|    0.333333

In [ ]:
grouped_amzn_stock_twts_df.count()

364

In [59]:
# Rounding off the values using ceil
from pyspark.sql.functions import ceil

# Assume `df` is a PySpark DataFrame with a column `col` that contains values to be rounded up
grouped_amzn_stock_twts_df = grouped_amzn_stock_twts_df.withColumn("daywise_avg_replyCount", ceil(grouped_amzn_stock_twts_df.daywise_avg_replyCount))
grouped_amzn_stock_twts_df = grouped_amzn_stock_twts_df.withColumn("daywise_avg_retweetCount", ceil(grouped_amzn_stock_twts_df.daywise_avg_retweetCount))
grouped_amzn_stock_twts_df = grouped_amzn_stock_twts_df.withColumn("daywise_avg_likeCount", ceil(grouped_amzn_stock_twts_df.daywise_avg_likeCount))
grouped_amzn_stock_twts_df = grouped_amzn_stock_twts_df.withColumn("daywise_avg_quoteCount", ceil(grouped_amzn_stock_twts_df.daywise_avg_quoteCount))


In [60]:
company_id = {'google':1,'microsoft':2, 'tesla':3, 'amazon':4, 'apple':5}

In [61]:
from pyspark.sql.functions import lit
grouped_amzn_stock_twts_df = grouped_amzn_stock_twts_df.withColumn('company',lit(company_id['amazon']))

In [62]:
grouped_amzn_stock_twts_df.show(10)

+----------+----------------------------+-----------------------------+-----------------------------+----------------------------+----------------------------+----------------------+------------------------+---------------------+----------------------+-------+
|      date|weighted_avg_sentiment_score|daywise_median_positive_score|daywise_median_negative_score|daywise_positive_tweet_count|daywise_negative_tweet_count|daywise_avg_replyCount|daywise_avg_retweetCount|daywise_avg_likeCount|daywise_avg_quoteCount|company|
+----------+----------------------------+-----------------------------+-----------------------------+----------------------------+----------------------------+----------------------+------------------------+---------------------+----------------------+-------+
|2022-01-01|         0.21028435754189945|                       0.5719|                       0.2732|                           6|                           3|                     2|                       4|          

## Saving the file

In [63]:
# converting pyspark dataframe to pandas dataframe
DF = grouped_amzn_stock_twts_df.toPandas()

DF.to_csv('Preprocessed Data/amazon_preprocessed.csv', index = False)

**Note**: Similarly preprocessing is done for other companies like AMAZON, GOOGLE, TESLA, MICROSOFT, APPLE